# 💼 Motor de Optimización de Carteras v2.0 - Análisis Interactivo

Este notebook te permite explorar **paso a paso** todas las capacidades del motor de optimización:

1. **📥 Descarga de Datos:** Configura tus activos y descarga datos históricos
2. **📊 Estadísticas:** Analiza retornos, volatilidad, correlaciones
3. **🎯 Optimización:** Mínima Volatilidad y Máximo Sharpe (Libre vs Gestionada)
4. **📉 VaR/CVaR:** Análisis de riesgo con distribución t-Student
5. **⏮️ Backtesting:** Validación histórica (Active vs Passive)
6. **🔥 Stress Testing:** Escenarios extremos
7. **📄 Reportes:** Generación automática de documentación

---

**Ventajas del Notebook:**
- ✅ Ejecución celda por celda para entender cada paso
- ✅ Fácil experimentación con diferentes activos y parámetros
- ✅ Visualización inline de gráficos
- ✅ Modificación rápida sin editar el script principal

**Autor:** Senior Quant Developer  
**Última actualización:** 2026-02-13

In [ ]:
# Importar librerías necesarias
import sys
sys.path.append('../src')

from portfolio_engine import PortfolioOptimizer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Configurar matplotlib para visualización inline
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

print("✅ Librerías importadas correctamente")
print(f"📦 Versión de NumPy: {np.__version__}")
print(f"📦 Versión de Pandas: {pd.__version__}")

---

## 📥 MÓDULO 1: Configuración de Activos y Descarga de Datos

En esta sección defines:
- **Activos a analizar** (tickers de Yahoo Finance)
- **Tasa libre de riesgo** (para cálculo de Sharpe Ratio)
- **Período de datos** (automático: últimos 2 años)

### 🔧 Parámetros Personalizables:

In [ ]:
# ===================================================
# 1.1 DEFINIR ACTIVOS A ANALIZAR
# ===================================================
# Puedes modificar esta lista con cualquier ticker de Yahoo Finance
# Ejemplos:
#   - Argentina: GGAL.BA, YPFD.BA, BBAR.BA, ALUA.BA, PAMP.BA
#   - Tech USA: AAPL, MSFT, GOOGL, META, NVDA
#   - Bancos: JPM, BAC, GS, WFC
#   - Cripto: BTC-USD, ETH-USD
#   - ETFs: SPY, QQQ, IWM

tickers = [
    'GGAL.BA',      # Banco Galicia (Argentina)
    'YPFD.BA',      # YPF (Argentina)
    'ALUA.BA',      # Aluar (Argentina)
    'GOOGL',        # Google (Tech Global)
    'BTC-USD',      # Bitcoin (Cripto)
    'MSFT',         # Microsoft (Tech Global)
    'JPM'           # JPMorgan Chase (Banca Global)
]

# ===================================================
# 1.2 CONFIGURAR PARÁMETROS
# ===================================================
risk_free_rate = 0.05  # Tasa libre de riesgo: 5% anual (default)
# Puedes cambiar a 0.03 (más conservador) o 0.07 (bonos alto rendimiento)

# ===================================================
# 1.3 INICIALIZAR OPTIMIZADOR
# ===================================================
optimizer = PortfolioOptimizer(tickers, risk_free_rate=risk_free_rate)

print(f"✅ Optimizador inicializado con {len(tickers)} activos")
print(f"📊 Activos: {', '.join(tickers)}")
print(f"💰 Tasa libre de riesgo: {risk_free_rate*100:.1f}%")

### 1.4 Descarga de Datos Históricos

El motor descarga automáticamente los **últimos 2 años** de datos desde Yahoo Finance.

In [ ]:
# Descargar datos históricos
data = optimizer.download_data()

print(f"\n📊 Datos descargados:")
print(f"   - Período: {data.index[0].strftime('%Y-%m-%d')} a {data.index[-1].strftime('%Y-%m-%d')}")
print(f"   - Días de trading: {len(data)}")
print(f"   - Activos: {len(data.columns)}")

# Visualizar últimas 5 filas
print("\n📈 Precios de Cierre (últimos 5 días):")
data.tail()

---

## 📊 MÓDULO 2: Estadísticas Descriptivas

Análisis de retornos, volatilidad, Sharpe Ratio y correlaciones.

In [ ]:
# Calcular estadísticas básicas
stats = optimizer.calculate_statistics()

print("\n📊 ESTADÍSTICAS INDIVIDUALES POR ACTIVO:")
print("="*80)
print(stats)

print("\n📈 Interpretación:")
print(f"  - Mejor Sharpe Ratio: {stats['Sharpe Ratio'].idxmax()} ({stats['Sharpe Ratio'].max():.2f})")
print(f"  - Mayor volatilidad: {stats['Volatilidad Anualizada'].idxmax()} ({stats['Volatilidad Anualizada'].max():.2f})")
print(f"  - Menor volatilidad: {stats['Volatilidad Anualizada'].idxmin()} ({stats['Volatilidad Anualizada'].min():.2f})")

---

## 🎯 MÓDULO 3: Optimización de Carteras

Calculamos dos carteras óptimas:
1. **Mínima Volatilidad:** Minimiza el riesgo total
2. **Máximo Sharpe Ratio:** Maximiza retorno ajustado por riesgo

Además, comparamos dos enfoques:
- **Libre:** Sin restricciones (0-100% en cada activo)
- **Gestionada:** Con límites realistas por tipo de activo

In [ ]:
# ===================================================
# 3.1 OPTIMIZACIÓN LIBRE (Sin restricciones)
# ===================================================
print("🔄 Optimizando carteras SIN restricciones...")

# Mínima Volatilidad
min_vol_weights_free, min_vol_perf_free = optimizer.optimize_min_volatility(bounds_dict=None)

# Máximo Sharpe
max_sharpe_weights_free, max_sharpe_perf_free = optimizer.optimize_max_sharpe(bounds_dict=None)

print("\n✅ Optimización LIBRE completada")
print(f"   - Min Vol Sharpe: {min_vol_perf_free[2]:.3f}")
print(f"   - Max Sharpe: {max_sharpe_perf_free[2]:.3f}")

In [ ]:
# ===================================================
# 3.2 OPTIMIZACIÓN GESTIONADA (Con restricciones)
# ===================================================
# 🔧 PERSONALIZA AQUÍ TUS RESTRICCIONES:
# Ejemplo: {'GGAL.BA': (0.0, 0.20)} = Máximo 20% en GGAL

bounds_managed = {
    'GGAL.BA': (0.00, 0.20),   # Max 20% en activos argentinos individuales
    'YPFD.BA': (0.00, 0.20),
    'ALUA.BA': (0.00, 0.20),
    'BTC-USD': (0.00, 0.10),   # Max 10% en cripto
    'GOOGL': (0.15, 0.35),     # Min 15%, Max 35% en tech líder
    'MSFT': (0.10, 0.30),      # Min 10%, Max 30%
    'JPM': (0.05, 0.25)        # Min 5%, Max 25%
}

print("🔄 Optimizando carteras CON restricciones...")
print(f"   Política: Max 20% argentinos, Max 10% cripto, Min 15% tech\n")

# Optimizar con restricciones
min_vol_weights_managed, min_vol_perf_managed = optimizer.optimize_min_volatility(bounds_dict=bounds_managed)
max_sharpe_weights_managed, max_sharpe_perf_managed = optimizer.optimize_max_sharpe(bounds_dict=bounds_managed)

print("✅ Optimización GESTIONADA completada")
print(f"   - Min Vol Sharpe: {min_vol_perf_managed[2]:.3f}")
print(f"   - Max Sharpe: {max_sharpe_perf_managed[2]:.3f}")

### 3.3 Visualizar Frontera Eficiente

In [ ]:
# Generar gráfico de frontera eficiente
fig, (min_vol_weights, min_vol_perf), (max_sharpe_weights, max_sharpe_perf) = optimizer.plot_efficient_frontier()

print("✅ Frontera eficiente generada")
print("   Gráfico incluye: Frontera, Correlaciones, Composición, Covarianza")

---

## 📉 MÓDULO 4: Análisis de Riesgo (VaR/CVaR con t-Student)

Calculamos **Value at Risk** y **Conditional VaR** usando:
- **Simulación de Monte Carlo:** 10,000 escenarios
- **Distribución t-Student:** Captura eventos extremos (fat tails)
- **3 Escenarios:** Normal, Esperado (df estimado), Conservador (df=3)

In [ ]:
# ===================================================
# 4.1 CALCULAR VaR/CVaR (Cartera Gestionada)
# ===================================================
print("🔄 Calculando VaR/CVaR con distribución t-Student...")

risk_metrics = optimizer.calculate_var_cvar(
    max_sharpe_weights_managed,
    confidence_level=0.95,
    n_simulations=10000
)

print("\n✅ Análisis de riesgo completado")
print(f"\n📉 VaR/CVaR al 95% (Conservador, df=3):")
print(f"   - VaR (1 día): {risk_metrics['var_conservative_daily']:.2%}")
print(f"   - CVaR (1 día): {risk_metrics['cvar_conservative_daily']:.2%}")
print(f"   - VaR (anualizado): {risk_metrics['var_conservative_annual']:.2%}")
print(f"   - CVaR (anualizado): {risk_metrics['cvar_conservative_annual']:.2%}")

---

## ⏮️ MÓDULO 5: Backtesting (Validación Histórica)

Validamos la estrategia con datos históricos reales:
- **Período:** Último año
- **Active Management:** Rebalanceo mensual con comisiones (0.5%)
- **Passive (Buy-Hold):** Sin rebalanceo
- **Benchmark:** Comparación vs SPY (S&P 500)

In [ ]:
# ===================================================
# 5.1 EJECUTAR BACKTESTING
# ===================================================
# 🔧 PERSONALIZAR AQUÍ:
initial_capital = 1000000  # Capital inicial en USD
lookback_period = '1y'     # Período: '1y', '2y', '6mo'
benchmark = 'SPY'          # Benchmark: 'SPY', 'QQQ', 'BTC-USD'
rebalance = True           # True = Active, False = Passive only
commission = 0.005         # 0.5% por operación

print(f"🔄 Ejecutando backtesting...")
print(f"   - Capital inicial: ${initial_capital:,.0f}")
print(f"   - Período: {lookback_period}")
print(f"   - Benchmark: {benchmark}")

backtest_results = optimizer.run_backtest(
    weights=max_sharpe_weights_managed,
    initial_capital=initial_capital,
    lookback_period=lookback_period,
    benchmark_ticker=benchmark,
    rebalance=rebalance,
    commission_pct=commission
)

print("\n✅ Backtesting completado")
print(f"\n📈 Resultados (Active Management):")
print(f"   - Retorno Total: {backtest_results['total_return_active']:.2f}%")
print(f"   - Sharpe Ratio: {backtest_results['sharpe_ratio_active']:.2f}")
print(f"   - Max Drawdown: {backtest_results['max_drawdown_active']:.2f}%")
print(f"   - Comisiones pagadas: ${backtest_results['total_commissions']:,.2f}")
print(f"\n📊 Resultados (Passive Buy-Hold):")
print(f"   - Retorno Total: {backtest_results['total_return_passive']:.2f}%")
print(f"   - Sharpe Ratio: {backtest_results['sharpe_ratio_passive']:.2f}")
print(f"   - Max Drawdown: {backtest_results['max_drawdown_passive']:.2f}%")

---

## 🔥 MÓDULO 6: Stress Testing (Escenarios Extremos)

Simulamos el impacto de eventos extremos en el capital:
- **Crash Global:** S&P -20%, Tech -25%, BTC -40%
- **Crisis Argentina:** Activos locales -40%
- **Recuperación Agresiva:** Todos +20%

In [ ]:
# ===================================================
# 6.1 EJECUTAR STRESS TESTING
# ===================================================
print("🔄 Ejecutando stress testing...")

stress_results = optimizer.run_stress_test(
    weights=max_sharpe_weights_managed,
    capital=initial_capital
)

print("\n✅ Stress Testing completado")
print(f"\n💥 Escenarios Extremos:")
for scenario, data in stress_results.items():
    print(f"\n   {scenario}:")
    print(f"      - Capital final: ${data['final_capital']:,.0f}")
    print(f"      - Impacto: {data['impact_pct']:.2f}%")
    print(f"      - Pérdida/Ganancia: ${data['total_impact']:,.0f}")

---

## 📄 MÓDULO 7: Generación de Reportes

Genera reportes técnicos en Markdown con todo el análisis.

In [ ]:
# ===================================================
# 7.1 GENERAR TODOS LOS REPORTES
# ===================================================
print("🔄 Generando reportes...")

# Reporte de Optimización
optimizer.generate_report(
    min_vol_weights_managed,
    max_sharpe_weights_managed,
    risk_metrics
)

# Reporte de Backtesting
optimizer.generate_backtest_report(backtest_results)

# Reporte de Stress Testing
optimizer.generate_stress_test_report(
    stress_results,
    max_sharpe_weights_managed,
    initial_capital
)

print("\n✅ Reportes generados en carpeta outputs/:")
print("   - reporte_portfolio.md")
print("   - reporte_backtesting.md")
print("   - reporte_stress_test.md")
print("   - efficient_frontier.png")
print("   - backtest_results.png")
print("   - stress_test.png")

---

## 📊 RESUMEN EJECUTIVO

Un resumen rápido de todos los resultados:

In [ ]:
print("="*80)
print(" "*25 + "RESUMEN EJECUTIVO")
print("="*80)

print(f"\n🎯 CARTERA ÓPTIMA (Máximo Sharpe Gestionada):")
print("-"*80)
for ticker, weight in zip(optimizer.tickers, max_sharpe_weights_managed):
    print(f"   {ticker:12s}: {weight:6.2%}")

print(f"\n📈 MÉTRICAS DE CARTERA:")
print("-"*80)
print(f"   Retorno Anualizado:     {max_sharpe_perf_managed[0]:.2%}")
print(f"   Volatilidad Anualizada: {max_sharpe_perf_managed[1]:.2%}")
print(f"   Sharpe Ratio:           {max_sharpe_perf_managed[2]:.3f}")

print(f"\n📉 RIESGO (VaR/CVaR al 95%, df=3):")
print("-"*80)
print(f"   VaR (1 día):            {risk_metrics['var_conservative_daily']:.2%}")
print(f"   CVaR (1 día):           {risk_metrics['cvar_conservative_daily']:.2%}")
print(f"   VaR (anualizado):       {risk_metrics['var_conservative_annual']:.2%}")

print(f"\n⏮️ BACKTESTING (1 año):")
print("-"*80)
print(f"   Retorno Active:         {backtest_results['total_return_active']:.2f}%")
print(f"   Retorno Passive:        {backtest_results['total_return_passive']:.2f}%")
print(f"   Sharpe Active:          {backtest_results['sharpe_ratio_active']:.2f}")
print(f"   Max Drawdown Active:    {backtest_results['max_drawdown_active']:.2f}%")
print(f"   Rebalanceo valió la pena: {'SÍ ✅' if backtest_results['rebalance_worthwhile'] else 'NO ❌'}")

print(f"\n🔥 STRESS TESTING:")
print("-"*80)
worst_scenario = min(stress_results.items(), key=lambda x: x[1]['impact_pct'])
best_scenario = max(stress_results.items(), key=lambda x: x[1]['impact_pct'])
print(f"   Peor escenario:         {worst_scenario[0]} ({worst_scenario[1]['impact_pct']:.2f}%)")
print(f"   Mejor escenario:        {best_scenario[0]} ({best_scenario[1]['impact_pct']:.2f}%)")

print("\n" + "="*80)
print(" "*20 + "✅ ANÁLISIS COMPLETADO")
print("="*80)
print("\n📂 Consulta los reportes detallados en outputs/")

---

## 💡 Notas Finales y Consejos de Uso

### 🔧 Parámetros Clave que Puedes Modificar:

1. **Activos (Celda 3):**
   - Cambia `tickers` para analizar diferentes activos
   - Asegúrate de que los tickers sean válidos en Yahoo Finance

2. **Restricciones (Celda 10):**
   - Modifica `bounds_managed` para ajustar límites por activo
   - Ejemplo: `{'BTC-USD': (0.0, 0.05)}` = Máximo 5% en Bitcoin

3. **Tasa Libre de Riesgo (Celda 3):**
   - Ajusta `risk_free_rate` según el contexto económico
   - Default: 5% (bonos del Tesoro de EE.UU.)

4. **Parámetros de Backtesting (Celda 16):**
   - `initial_capital`: Capital inicial
   - `lookback_period`: '1y', '2y', '6mo', etc.
   - `benchmark`: 'SPY', 'QQQ', 'BTC-USD'
   - `commission`: 0.005 (0.5%), ajusta según tu broker

### 📖 Recursos Adicionales:

- **INSTRUCCIONES.md:** Guía detallada de uso y personalización
- **README.md:** Documentación del proyecto
- **outputs/:** Carpeta con reportes y gráficos generados

### ⚠️ Advertencias:

- Los resultados históricos **no garantizan** rendimientos futuros
- Las optimizaciones asumen correlaciones estables (pueden cambiar en crisis)
- Considera costos de transacción, impuestos y slippage en implementación real
- Para Argentina: monitorea riesgo país, controles cambiarios y políticas macro

---

**¿Dudas o mejoras?** Consulta la documentación completa o experimenta con diferentes configuraciones en las celdas anteriores. ¡Buena suerte con tu análisis! 🚀